## Workshop 1b: Mixed Integer Linear Program Exercise

### 1. Installing and Importing Packages 

We first need to pull in all the packages we will be using. Pyomo is a Python-based, open-source optimization modelling language with a diverse set of optimization capabilities. For more information, see the Pyomo [documentation](https://pyomo.readthedocs.io/en/stable/).

In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
import numpy as np
from ipywidgets import FloatSlider, interact, widgets
import platform
from IPython.display import display

# Solver setup for Windows or Linux
def setup_solver():
    os_name = platform.system()
    if os_name == "Windows":
        return "solver/ipopt.exe", "solver/cbc.exe"
    elif os_name == "Linux":
        !chmod +x "solver/ipopt"
        return "solver/ipopt", "solver/cbc"

ipopt_executable, cbc_executable = setup_solver()

### 2. MILP with Binary Variables

A key feature of MILP formulations is the potential to have binary variables, often represented by $y$ as in this example. We can use binary variable to model semi-continous variables, for example taking $x$ in this example:

$$
x = 0 \: \vee \: 10 \leq x \leq 20
$$

Can be reformulated using a binary variable, $y$:

$$
10y \leq x, \quad 20y \leq x
$$

$$
y \in \{0, 1\}
$$


#### a. Example Modelling Fixed-Cost Objectives
$$
\text{minimize} \: f_1(x_1) + f_2(x_2)
$$

$$
\text{s.t.} \quad x_1 + x_2 \leq 8
$$

$$
0 \leq x_1 \leq 3, \quad 0 \leq x_2 \leq 8
$$

where:

$$
f_1(x_1) = 
\begin{cases} 
150 + 7x_1 & \text{if } x_1 > 0 \\ 
0 & \text{otherwise}
\end{cases}
$$

$$
f_2(x_2) = 
\begin{cases} 
110 + 9x_2 & \text{if } x_2 > 0 \\ 
0 & \text{otherwise}
\end{cases}
$$

In [9]:
# Create a Pyomo model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(bounds=(0,3), within=NonNegativeReals)
model.x2 = Var(bounds=(0, 8), within=NonNegativeReals)
model.y1 = Var(within=Binary)
model.y2 = Var(within=Binary)

# Objective function
model.obj = Objective(expr=7 * model.x1 + 9 * model.x2 + 150*model.y1 + 110*model.y2, sense=minimize)

# Constraints
model.con1 = Constraint(expr=model.x1 + model.x2 <= 8)
model.con2 = Constraint(expr=model.x1 <=  3*model.y1)
model.con3 = Constraint(expr=model.x2 <=  8*model.y2)

# Create a solver
solver = SolverFactory('cbc', executable=cbc_executable)

# Solve the model
solver.solve(model)

# Display the results
print(f"x1: {model.x1():.2f}")
print(f"x2: {model.x2():.2f}")
print(f"y3: {(model.y1())}")
print(f"y3: {(model.y2())}")

x1: 0.00
x2: 0.00
y3: 0.0
y3: 0.0


#### b. Example Problem: Semi-Continuous Blending Variables

$x_3$ in this problem is semi-continuous, we can reformulate using binary variables.
$$
\text{minimize} \quad 18x_1 + 3x_2 + 9x_3
$$

$$
\text{s.t.} \quad 2x_1 + x_2 + 7x_3 \leq 150
$$

$$
0 \leq x_1 \leq 60, \quad 0 \leq x_2 \leq 30
$$

$$
x_3 = 0 \: \vee \:  10 \leq x_3 \leq 20
$$

$$
x_1, x_2, x_3 \in \mathbb{R}^+
$$

In optimization problems, particularly when dealing with binary and continuous variables, careful formulation is essential to maintain problem tractability. Although a problem could be reformulated with additional binary variables leading to bilinear terms in the objective function, such a formulation would convert the problem into a MINLP problem. MINLPs can be significantly harder to solve due to their non-convex nature and the potential for multiple local optima.

To avoid this complexity, we can introduce a binary variable $y_3$ and an additional continuous variable $z_3$. This allows us to reformulate the original problem as a Mixed-Integer Linear Programming (MILP) problem.

$$
\text{minimize} \quad 18x_1 + 3x_2 + 9z_3
$$

$$
\text{s.t.} \quad 2x_1 + x_2 + 7z_3 \leq 150
$$

$$
0 \leq x_1 \leq 60, \quad 0 \leq x_2 \leq 30
$$

$$
10y_3 \leq z_3 \leq 20y_3
$$

$$
x_3 + 20(y_3-1) \leq z_3 \leq x_3 + 10(y_3-1)
$$

$$
x_1, x_2, x_3, z_3 \in \mathbb{R}^+
$$

$$
y \in \{0, 1\}
$$


In [3]:
# Create a Pyomo model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(bounds=(0, 60), within=NonNegativeReals)
model.x2 = Var(bounds=(0, 30), within=NonNegativeReals)
model.x3 = Var(bounds=(None,None), within=NonNegativeReals)
model.z3 = Var(within=NonNegativeReals)
model.y3 = Var(within=Binary)

# Objective function
model.obj = Objective(expr=18 * model.x1 + 3 * model.x2 + 9*model.z3, sense=minimize)

# Constraints
model.con1 = Constraint(expr=2*model.x1 + model.x2 + 7*model.x3 <= 150)
model.con2 = Constraint(expr=model.x3 +20*(model.y3-1)<= model.z3)
model.con3 = Constraint(expr= model.z3 <= model.y3 + 10*(model.y3-1))

# Create a solver
solver = SolverFactory('cbc', executable=cbc_executable)

# Solve the model
solver.solve(model)

# Display the results
print(f"Objective Value: {(model.obj())}\n")
print(f"x1: {model.x1():.2f}")
print(f"x2: {model.x2():.2f}")
print(f"x3: {model.x3():.2f}")
print(f"y3: {(model.y3())}")
print(f"z3: {(model.z3())}")


Objective Value: 0.0

x1: 0.00
x2: 0.00
x3: 0.00
y3: 1.0
z3: 0.0


#### c. Relaxing MILP to LPs

Problem 1: Setting $y_3 = 0$ 

$$
\text{minimize} \quad 18x_1 + 3x_2
$$

$$
\text{s.t.} \quad 2x_1 + x_2 \leq 150
$$

$$
0 \leq x_1 \leq 60, \quad 0 \leq x_2 \leq 30
$$

$$
x_1, x_2, x_3 \in \mathbb{R}^+, \quad y \in \{0, 1\}
$$

$$
\text{Since} \: y_3=0: z_3=0, x_3 = 0
$$

---

Problem 2: Setting $y_3 = 1$ 
$$
\text{minimize} \quad 18x_1 + 3x_2 + 9z_3
$$

$$
\text{s.t.} \quad 2x_1 + x_2 + 7z_3 \leq 150
$$

$$
0 \leq x_1 \leq 60, \quad 0 \leq x_2 \leq 30
$$

$$
10 \leq z_3 \leq 20
$$

$$
x_3 \leq z_3 \leq x_3
$$

$$
x_1, x_2, x_3 \in \mathbb{R}^+, \quad y \in \{0, 1\}
$$

$$
\text{Since} \: y_3=1: \: 10 \leq z_3 \leq 20, \: x_3 = z_3
$$

### 3. MILP Case Study

![image (2).png](<images/reactor_milp.png>)

In [4]:
# Create the model
model = ConcreteModel()

## Decision variables - quantity of A sent to reactor i
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)
model.x3 = Var(domain=NonNegativeReals)

# Binary decision variables - used to calculate fixed cost of reactor i
model.y1 = Var(domain=Binary)
model.y2 = Var(domain=Binary)
model.y3 = Var(domain=Binary)

# Cost Parameters (hint - once solved, play with c_x1 and c_x2)
c_y1, c_y2, c_y3 = 80, 54, 27
c_x1, c_x2, c_x3 = 35, 30, 25

# Objective function (minimize cost)
model.cost = Objective(
    expr=c_y1*model.y1 + c_x1*model.x1 + c_y2*model.y2 + c_x2*model.x2 + c_y3*model.y3 + c_x3*model.x3,
)

## Constraints

# Material Balance: to achieve 10 kg/hr of B
model.material_balance = Constraint(expr=0.8*model.x1 + 0.667*model.x2 + 0.555*model.x3 == 10)

# Feed limit: only 15 kg/hr of A can be fed
model.feed_limit = Constraint(expr=model.x1 + model.x2 + model.x3 <= 15)

# Selection constraints - linking binary variable to whether reactor i is used or not
model.selection1 = Constraint(expr=model.x1 - 15*model.y1 <= 0)
model.selection2 = Constraint(expr=model.x2 - 15*model.y2 <= 0)
model.selection3 = Constraint(expr=model.x3 - 15*model.y3 <= 0)

# Solve the model
solver = SolverFactory('cbc', executable=cbc_executable)
solver.solve(model)

# Display results
print(f"x1: {model.x1():.3f}, y1: {model.y1():.3f}")
print(f"x2: {model.x2():.3f}, y2: {model.y2():.3f}")
print(f"x3: {model.x3():.3f}, y3: {model.y3():.3f}")
print(f"Total cost: {model.cost():.3f}")


x1: 0.000, y1: 0.000
x2: 14.993, y2: 1.000
x3: 0.000, y3: 0.000
Total cost: 503.775


#### Extension Challenge: 
Can you add an additional constraint that enforces that at least two reactors must be used?